# Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Stocastic Differential Equations

## Asset Price

In [30]:
class AssetPrice:
    """
    This is a class for Asset Price ODE. 
    It contains methods for finding exact solution, plotting and exat mean hitting time.
    Arguments:
    a: Lower bound
    b: Upper bound
    mu: mu
    sigma: sigma
    """
    def __init__(self, lower_b, upper_b, mu, sigma):
        self.lower_b = lower_b
        self.upper_b = upper_b
        self.mu = mu
        self.sigma = sigma
        
    def f(self,x):
        """
        Function of f(x) in asset price stochastic differential equation (deterministic part)
        """
        return self.mu * x
    
    def g(self,x):
        """
        Function of g(x) in asset price stochastic differential equation (stochastic part)
        """
        return self.sigma * x
    
    def dg(self,x):
        """
        Function of g'(x) in asset price stochastic differential equation (stochastic part)
        """
        return self.sigma
  

## Asset Price Interest Rate

In [3]:
class AssetPriceInterestRate:
    def __init__(self, lam, mu, sigma):
        self.lam = lam
        self.mu = mu
        self.sigma = sigma
        
    
    def f(self,x):
        return self.lam*(self.mu - x)
    
    def g(self,x):
        return self.sigma*np.sqrt(x)
    

## Opinion Polls

In [4]:
class OpinionPolls:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def f(self,x):
        return -self.mu*(x/(1-x**2))
        
    def g(self,x):
        return self.sigma

## Population Dynamic 

In [5]:
class PopulationDynamic:
    def __init__(self, K, r, beta):
        self.K = K
        self.r = r
        self.beta = beta
        
    def f(self,x):
        return self.r*x*(self.K - x)
    
    def g(self,x):
        return self.beta*x

## Epidemic Model

In [6]:
class EpidemicModel:
    
    def __init__(self, p, B, beta, alpha, rho, C):
        self.p = p
        self.B = B
        self.beta = beta
        self.alpha = alpha
        self.rho = rho 
        self.C = C
    
    def f(self, x):
        return (self.p -1)*self.B*x + (self.beta*self.C - self.alpha)*(1 - x)*x
    
    def g(self, x):
        return self.p*self.C*(1-x)*x

## Political Opinion

In [7]:
class PoliticalOpinion:
    def __init__(self, r, G, eps):
        self.r = r
        self.G = G
        self.eps = eps
        
    def f(self,x):
        return self.r*(self.G-x)
    
    def g(self,x):
        return np.sqrt(self.eps*x*(1-x))
    

## Double Well Potential

In [8]:
class DoubleWellPotential:
    def __init__(self, sigma):
        self.sigma = sigma
        
    def _V(self,x):
        return (x**2)*(x-2)**2
    
    def _dV(self,x):
        return 4*x*(x-2)*(x-1)
    
    def f(self,x):
        return -self._dV(x)
    
    def g(self,x):
        return self.sigma

In [9]:
class SimpleSDE:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
    
    def f(self, x):
        return self.mu
    
    def g(self, x):
        return self.sigma
    

# Methods

## EXPONENTIAL TIMESTEPPING WITH BOUNDARY TEST FOR SDEs 

### The exponential-Euler method with boundary test (1st Algorithm)

#### Assumptions: $f(X_t) = \mu$ and $\sigma(X_t)=\epsilon$

In [10]:
class ExponentialTimestepping:
    def __init__(self, rate):
        self.rate = rate
    
    def F(self, Xt, f, g):
        return f(Xt) / g(Xt)**2
        
    def N(self, Xt, g):
        return np.sqrt((2*self.rate / g(Xt)**2) + self.F(Xt)**2)
    
    
    def compute_MHT(self, X0, dt, f, g , num_itr, lower_b=None, upper_b=None):
        if lower_b is None and upper_b is None:
            assert("Please provide a boundary value")  
        if lower_a is None:
            lower_a = -np.inf
        if uuper_b is None:
            lower_b = np.inf
            
        t_exit = []
        
        for i in range(num_itr):
            self.breaked = 0 
            tn = 0
            Xn = X0
            while Xn > lower_b and Xn < upper_b:
                v = np.random.uniform()
                p = -log(v)
                u = np.random.uniform()
                sign = np.sign(0.5 * (1 + self.F(Xt)/self.N(Xt)) - u)
                Xt_1 = Xt + (sign*p)/(self.N(Xt)-sign*self.F(Xt))
                
                w = np.random.uniform()
                if Xn_1 < lower_b or w < np.exp(-2*self.N(b-max(Xt, Xn_1))) or Xnew > upper_b or w < np.exp(-2*self.N(min(Xt, Xn_1)-b)):
                    self.breaked += 1
                    break
                    
                tn += dt
                Xn = Xn_1
        
        tmean = np.mean(t_exit)
        tstd = np.std(t_exit)

        cileft = t_exit, tmean - 1.96*tstd/np.sqrt(npaths)
        ciright = t_exit, tmean + 1.96*tstd/np.sqrt(npaths)
        
        return tmean, tstd, cileft, ciright
    
    def plot(self,t_exit):
        histogram,bins = np.histogram(t_exit,bins=20,range=[0,20])
        midx = (bins[0:-1]+bins[1:])/2
        plt.bar(midx,histogram,label='Test')
        plt.show()
    

### The exponential-V method with boundary test(2nd Algorithm)

#### Assumptions: $\sigma(x) = \epsilon$ but $f(x)$ is a function of $x$

In [32]:
class ExponentialVTimestepping:
    def __init__(self, rate):
        self.V = None
        self.rate = rate
        
    
    def nu(self, Xt, g):
        return np.sqrt(2*self.rate / g(Xt)**2)
    

    def compute_MHT(self, X0, dt, f, g , num_itr, lower_b=None, upper_b=None):
        if self.V is None:
            assert("Please provide value for V")
            
        if lower_b is None and upper_b is None:
            assert("Please provide a boundary value")  
        if lower_a is None:
            lower_a = -np.inf
        if uuper_b is None:
            lower_b = np.inf
            
        t_exit = []
        
        for i in range(num_itr):
            self.breaked = 0 
            tn = 0
            Xn = X0
            while Xn > lower_b and Xn < upper_b:
                nu = self.nu(Xt, g)
                v = np.random.uniform()
                p = -log(v)
                u = np.random.uniform()
                sign = np.sign(0.5*(1 + (nu**(-2)) * (g(Xt)**(-2))*f(Xt)) - u)
                Xt_1 = Xt + (nu**(-1))*sign*(p - (g(Xt)**(-2))*(self.V(Xt + (nu**(-1))*sign*p) - self.V(Xt)))
                                 
                w = np.random.uniform()
                nu = self.nu(Xt, g)
                if Xt_1 < lower_b or w < np.exp(-2*nu(min(Xt, Xt_1)-b)) or Xt_1 > upper_b or w < np.exp(-2*nu(b - max(Xt, Xt_1))):
                    self.breaked += 1
                    break
                
                tn += dt
                Xn = Xn_1
                
        tmean = np.mean(t_exit)
        tstd = np.std(t_exit)

        cileft = tmean - 1.96*tstd/np.sqrt(npaths)
        ciright = tmean + 1.96*tstd/np.sqrt(npaths)
        
        return t_exit, tmean, tstd, cileft, ciright
    
    
    def plot(self,t_exit):
        histogram,bins = np.histogram(t_exit,bins=20,range=[0,20])
        midx = (bins[0:-1]+bins[1:])/2
        plt.bar(midx,histogram,label='Test')
        plt.show()
    

-----------------------------------------------------------

## Absorbing boundaries and optimal stopping in a stochastic differential equation

#### Assumtions $g(x)=\sigma$

In [13]:
class EulerMaryamaBoundaryCheck:
    def __init__(self):
        self.breaked = 0
        self.thres_coeff = 5
        pass
    
    def P_hit(self, Xn,Xn_1,dt,bound,D, df):
        return np.exp(-df(bound)/(2*D*(np.exp(2*dt*df(bound))-1))*(Xn_1-bound+(Xn-bound)*np.exp(dt*df(bound))-f(bound)/df(bound))**2 + (bound - (Xn + dt*(f(Xn)+f(Xn_1))/2))**2/4*D*dt)
    
    
    def compute_MHT(self, X0, dt, f, g, df, num_itr, lower_b=None, upper_b=None):
        if lower_b is None and upper_b is None:
            assert("Please provide a boundary value")  
        if lower_a is None:
            lower_a = -np.inf
        if uuper_b is None:
            lower_b = np.inf
            
        t_exit = []
        
        for i in range(num_itr):
            self.breaked = 0 
            tn = 0
            Xn = X0
            while Xn > lower_b and Xn < upper_b:
                Rn = np.random.randn(1)
                Xn_1 = Xt + dt*f(Xt) + np.sqrt(dt)*Rn*g(Xt)
                D = (g(Xt)**2)/2
                if Xn-a<self.thres_coeff*dt or b-Xn<self.thres_coeff*dt:
                    prob = self.P_hit(Xn,Xn_1,dt,a,D, df)
                    if prob>np.random.uniform(0,1) or prob>np.random.uniform(0,1):
                        self.breaked += 1
                        break
                        
                tn += dt
                Xn = Xn_1
        
        tmean = np.mean(t_exit)
        tstd = np.std(t_exit)

        cileft = tmean - 1.96*tstd/np.sqrt(npaths)
        ciright = tmean + 1.96*tstd/np.sqrt(npaths)
        
        return tmean, tstd, cileft, ciright
    
    def plot(self,t_exit):
        histogram,bins = np.histogram(t_exit,bins=20,range=[0,20])
        midx = (bins[0:-1]+bins[1:])/2
        plt.bar(midx,histogram,label='Test')
        plt.show()
        


    

_______________________________________

## Adaptive Timestep

In [14]:
class AdaptiveTimestep:
    def __init__(self):
        pass
    
    def naive_adaption(self, lower_b, upper_b, dt, x):
        """
        A naive method to adapt timestep based on the distance to the boundaries
        """
        epsilon = 1e-10
        dist = min(abs(x - lower_b), abs(x - upper_b))
        dt_new = (abs(np.tanh(dist))+epsilon) * dt
        return dt_new
    
    
    def adapt_time_EM(self, lower_b, upper_b, x, f, g, dt):
        min_dt = 0.0000001
        k = 1.96 # set to 1.96 for 95% credible interval of gaussian distribution
        
        f_res = f(x)
        g_res = g(x)
        
        if f_res == 0 and g(x) == 0:
            return dt
        elif g_res == 0:
            dt_new = min((upper_b-x)/f_res, (x-lower_b)/f_res)
            return min(dt, max(dt_new, min_dt))
        elif f_res == 0:
            if g_res > 0:
                dt_new = ((upper_b-x)/(k*g_res))**2
                return min(dt, max(dt_new, min_dt))
            else:
                dt_new = ((x-lower_b)/(k*g_res))**2
                return min(dt, max(dt_new, min_dt))
        elif f_res > 0:
            t1 = np.sqrt( ((upper_b-x)/f_res) + ((k*g_res)/(2*f_res))**2) - (k*g_res)/(2*f_res)
            t2 = np.sqrt( ((upper_b-x)/f_res) + ((k*g_res)/(2*f_res))**2) + (k*g_res)/(2*f_res)
            dt_new = min(t1**2, t2**2)
            return min(dt, max(dt_new, min_dt))
        
        t1 = np.sqrt( ((x-lower_b)/f_res) + ((k*g_res)/(2*f_res))**2) - (k*g_res)/(2*f_res)
        t2 = np.sqrt( ((x-lower_b)/f_res) + ((k*g_res)/(2*f_res))**2) + (k*g_res)/(2*f_res)
        dt_new = min(t1**2, t2**2)
        return min(dt, max(dt_new, min_dt))
    
    
    
    def _f2(self,x,f,g,dg,k):
        return f(x) + 0.5*g(x)*dg(x)*((k**2)-1)
        
    def adapt_time_Mils(self, lower_b, upper_b, x, f, g, dg, dt):
        min_dt = 0.0000001
        k = 1.96 # set to 1.96 for 95% credible interval of gaussian distribution
        
        f2_res = self._f2(x,f,g,dg,k)
        g_res = g(x)
        
        if f2_res == 0 and g_res == 0:
            return dt
        
        elif g_res == 0:
            dt_new = min((upper_b-x)/f2_res, (x-lower_b)/f2_res)
            return min(dt, max(dt_new, min_dt))
        
        elif f2_res == 0:
            if g_res > 0:
                dt_new = ((upper_b-x)/(k*g_res))**2
                return min(dt, max(dt_new, min_dt))
            else:
                dt_new = ((x-lower_b)/(k*g_res))**2
                return min(dt, max(dt_new, min_dt))
                          
        elif f2_res > 0:
            t1 = np.sqrt( ((upper_b-x)/f2_res) + ((k*g_res)/(2*f2_res))**2) - (k*g_res)/(2*f2_res)
            t2 = np.sqrt( ((upper_b-x)/f2_res) + ((k*g_res)/(2*f2_res))**2) + (k*g_res)/(2*f2_res)
            dt_new = min(t1**2, t2**2)
            return min(dt, max(dt_new, min_dt))
        
        t1 = np.sqrt( ((x-lower_b)/f2_res) + ((k*g_res)/(2*f2_res))**2) - (k*g_res)/(2*f2_res)
        t2 = np.sqrt( ((x-lower_b)/f2_res) + ((k*g_res)/(2*f2_res))**2) + (k*g_res)/(2*f2_res)
        dt_new = min(t1**2, t2**2)
        return min(dt, max(dt_new, min_dt))
    
    
    
    def compute_MHT_EM(self, X0, dt, num_itr, f, g, lower_b, upper_b):
        """
        Method that approxiamte a solution using Euler-Maruyama method
        
        Arguments:
        f: F(x)
        g: g(x)
        
        Return: List containing Mean, STD, Confidence interval Left, Confidence interval Right
        """
        # TODO: Add threshold for situation when the loop goes forever
        t_exit = np.zeros(npaths)
        
        adapt_timestep = self.adapt_time_EM()
        for i in tqdm(range(num_itr)):
            X = X0
            t = 0
            while X > lower_b and X < upper_b:
                dt_new_EM = adapt_timestep(lower_b=lower_b, upper_b=upper_b, x=X, f=f, g=g, dt=dt)
                dW = np.sqrt(dt_new_EM)*np.random.randn()
                X = X + dt_new_EM*f(X) + g(X)*dW
                t += dt_new_EM

            t_exit[i] = t


        
            
        tmean = np.mean(t_exit)
        tstd = np.std(t_exit)

        cileft = tmean - 1.96*tstd/np.sqrt(num_itr)
        ciright = tmean + 1.96*tstd/np.sqrt(num_itr)
        
        return t_exit, tmean, tstd, cileft, ciright
    
    
    
    def MHT_Milstein(self,X0,dt,num_itr,f,g,dg,lower_b, upper_b,adaptive= False):
        """
        Method that approxiamte a solution using Euler-Maruyama method
        
        Arguments:
        f: F(x)
        g: g(x)
        
        Return: List containing Mean, STD, Confidence interval Left, Confidence interval Right
        """
        # TODO: Add threshold for situation when the loop goes forever
        t_exit = np.zeros(npaths)
        
        

        adapt_timestep = self.adapt_time_Mils
        for i in tqdm(range(num_itr)):
            X = X0
            t = 0
            # TODO: pass new dt to adapt_timestep
            while X > lower_b and X < upper_b:
                dt_new_EM = adapt_timestep(lower_b=lower_b, upper_b=upper_b, x=X, f=f, g=g, dg=dg, dt=dt)
                dW = np.sqrt(dt_new_EM)*np.random.randn()
                X = X + dt_new_EM*f(X) + g(X)*dW + 0.5 * g(X)*dg(X)*(dW**2 - dt_new_EM)
                t += dt_new_EM

            t_exit[i] = t


            
        tmean = np.mean(t_exit)
        tstd = np.std(t_exit)

        cileft = tmean - 1.96*tstd/np.sqrt(num_itr)
        ciright = tmean + 1.96*tstd/np.sqrt(num_itr)
        
        return t_exit, tmean, tstd, cileft, ciright
    


    def plot(self,t_exit):
        histogram,bins = np.histogram(t_exit,bins=20,range=[0,20])
        midx = (bins[0:-1]+bins[1:])/2
        plt.bar(midx,histogram,label='Test')
        plt.show()
        

In [21]:
class EM_Milstein:
    def __init__(self):
        pass
    
    def compute_MHT_EM(self,X0 , dt, num_itr, f, g, lower_b, upper_b):
        t_exit = np.zeros(npaths)
        for i in tqdm(range(npaths)):
            X = X0
            t = 0
            while X > lower_b and X < upper_b:
                dW = np.sqrt(dt) * np.random.randn()
                X = X + dt*f(X) + g(X)*dW
                t += dt

            t_exit[i] = t - 0.5*dt
                
    
        tmean = np.mean(t_exit)
        tstd = np.std(t_exit)

        cileft = tmean - 1.96*tstd/np.sqrt(num_itr)
        ciright = tmean + 1.96*tstd/np.sqrt(num_itr)
        
        return t_exit, tmean, tstd, cileft, ciright
    
    def compute_MHT_Milstein(self, X0, dt, num_itr, f, g, dg, lower_b, upper_b):
        t_exit = np.zeros(npaths)
        for i in tqdm(range(npaths)):
            X = X0
            t = 0
            while X > LB and X < UB:
                dW = np.sqrt(dt) * np.random.randn()
                X = X + dt*f(X) + g(X)*dW + 0.5 * g(X)*dg(X)*(dW**2 - dt)
                t += dt

            t_exit[i] = t - 0.5*dt
            
        tmean = np.mean(t_exit)
        tstd = np.std(t_exit)

        cileft = tmean - 1.96*tstd/np.sqrt(num_itr)
        ciright = tmean + 1.96*tstd/np.sqrt(num_itr)

        return t_exit, tmean, tstd, cileft, ciright
    
    
    def plot(self,t_exit):
        histogram,bins = np.histogram(t_exit,bins=20,range=[0,20])
        midx = (bins[0:-1]+bins[1:])/2
        plt.bar(midx,histogram,label='Test')
        plt.show()

________________________

# Expariments

### Simulation Parametres and Mothod Instantiation

In [ ]:
num_itr = 5e4
exp_rate = 1000 # Lambda parameter for exponential timestepping methods

# Methods 
EM_Milstein = EM_Milstein() # Euler-maryama and Milstein method
AdaptiveTimestep = AdaptiveTimestep()
EulerMaryamaBoundaryCheck = EulerMaryamaBoundaryCheck()
ExponentialVTimestepping = ExponentialVTimestepping(rate=exp_rate)
ExponentialTimestepping = ExponentialTimestepping(rate=exp_rate)

### Simple SDE: $dX = \mu dt + \sigma dW, \quad X(0) = X_0$

In [ ]:
# Parameters
mu = 0.5
sigma = 1
X0 = 0
upper_b = 5
dt = 0.01

# New instance of SimpleSDE class
simpleSDE = simpleSDE(mu = mu, sigma = sigma)

## Asset Price: $dX = \mu X dt + \sigma X dW \qquad 0 < a < X_{0} < b$

In [31]:
# Parameters
mu = 0.1
sigma = 0.2
X0 = 1
upper_b = 2
lower_b = 0.2
exact_MHT = 7.6450

# Asset Price instance
AssetPrice = AssetPrice(lower_b=lower_b, upper_b=upper_b, mu=mu, sigma=sigma)
f = AssetPrice.f
g = AssetPrice.g
dg = AssetPrice.dg

# Run simulation
t_exit, tmean_EM, tstd_EM, cileft_EM, ciright_EM = EM(X0=X0,dt=dt,num_itr=num_itr, f=f, g=g, lower_b=lower_b,upper_b=upper_b)

#### Euler-Maruyama

#### Milstien

####  EXPONENTIAL TIMESTEPPING WITH BOUNDARY TEST FOR SDEs (The exponential-Euler method with boundary test) 

#### Absorbing boundaries and optimal stopping in a stochastic differential equation

#### Adaptive Timestep-Euler-Maruyama

________________

### Double Well Potential

#### Euler-Maruyama

#### Milstien

####  EXPONENTIAL TIMESTEPPING WITH BOUNDARY TEST FOR SDEs (The exponential-Euler method with boundary test) 

#### Absorbing boundaries and optimal stopping in a stochastic differential equation

#### Adaptive Timestep-Euler-Maruyama

#### Adaptive Timestep-Milstein

### Opinion Poll

#### Euler-Maruyama

#### Milstien

####  EXPONENTIAL TIMESTEPPING WITH BOUNDARY TEST FOR SDEs (The exponential-Euler method with boundary test) 

#### Absorbing boundaries and optimal stopping in a stochastic differential equation

#### Adaptive Timestep-Euler-Maruyama

#### Adaptive Timestep-Milstein